In [1]:
from glob import glob
import random
import math
from tqdm import tqdm

import numpy as np
import ipyplot
from PIL import Image
from sklearn.cluster import KMeans
from itertools import compress
import pickle
import pandas as pd
import cv2
import ipyplot
import gensim
from ast import literal_eval

from MoCoFeatureExtractor import MoCoFeatureExtractor

cuda


### Parameters and Utility methods for extracting patches

In [12]:
window_size = 288
stride = 144

n_clusters = 1000
word_format_string = '{:03d}'

walk_length = 10
walks_per_image = 100

cluster_patches_per_image = 24

image_scale = 1

feature_dim = 2048

cluster_file = f'clusters_{window_size}_{stride}_{n_clusters}.pkl'
image_cluster_grid_file = f'image_cluster_grids_{window_size}_{stride}_{n_clusters}.npy'
sequences_file = f'sequences_{window_size}_{stride}_{n_clusters}.csv'
doc2vec_file = f'doc2vec_{window_size}_{stride}_{n_clusters}.model'

cnn = MoCoFeatureExtractor(params_path='/home/ubuntu/moco_v2_800ep_pretrain.pth.tar')

image_files = glob("/home/ubuntu/dataset_1000/train/*/*.jpg")

image_id_to_class = dict([(f.split('/')[-1].split('.')[0], f.split('/')[-2]) for f in image_files])

def extract_windows(frame, pos, window_size):
    windows = np.empty((len(pos), window_size, window_size, 3), dtype=np.uint8)

    for i in range(len(pos)):
        windows[i] = extract_window(frame, pos[i], window_size)

    return windows


def extract_window(frame, pos, window_size):
    half_w = window_size/2.0

    top_left = [int(round(pos[0]-half_w)), int(round(pos[1]-half_w))]
    bottom_right = [top_left[0]+window_size, top_left[1]+window_size]

    return frame[top_left[0]:bottom_right[0], top_left[1]:bottom_right[1]]



def get_rad_grid(grid_pos, rad, grid_shape):

    top_left = (grid_pos[0]-rad, grid_pos[1]-rad)

    res = []

    for i in range(2*rad+1):
        p = (top_left[0]+i, top_left[1])
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)
 
    for i in range(2*rad+1):
        p = (top_left[0]+i, top_left[1]+(2*rad))
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)

    for i in range(2*rad-1):
        p = (top_left[0], top_left[1]+(i+1))
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)

    for i in range(2*rad-1):
        p = (top_left[0]+(2*rad), top_left[1]+(i+1))
        if p[0] >= 0 and p[1] >= 0 and p[0] < grid_shape[0] and p[1] < grid_shape[1]:
            res.append(p)

    return res



def next_pos(salient_grid_positions, grid_shape, current_position):
    
    if current_position is not None:

        rad_grid = get_rad_grid(current_position, 1, grid_shape)

        # print('rad_grid', current_position, rad_grid)
        
        if len(rad_grid) == 0:
            print("frame empty?")
            
        else:
            random.shuffle(rad_grid)
            for loc in rad_grid:
                if loc in salient_grid_positions:
                    return loc
    
    return random.sample(salient_grid_positions,1)[0]

### Sample and cluster patches
This simply uses a fixed grid system. Future patch sampling methods could incorporate an intrest point detector.

In [5]:
X = []


for idx, image_file in tqdm(enumerate(image_files), total=len(image_files)):

    pil_image = Image.open(image_file).convert('RGB')
    pil_image = pil_image.resize((int(round(pil_image.size[0] * image_scale)), int(round(pil_image.size[1] * image_scale))))
    image = np.array(pil_image)

    if image.shape[0] < window_size * 2 or image.shape[1] < window_size * 2:
        continue

    margin = window_size-stride
    grid_shape = (math.floor((image.shape[0] - margin) / stride), math.floor((image.shape[1] - margin) / stride))
    offsets = (round((image.shape[0] - grid_shape[0] * stride)/2), round((image.shape[1] - grid_shape[1] * stride)/2))

    points = [(offsets[0]+y*stride+stride/2,offsets[1]+x*stride+stride/2) for y in range(grid_shape[0]) for x in range(grid_shape[1])]

    if len(points) > cluster_patches_per_image:
        points = random.sample(points, cluster_patches_per_image)

    patches = extract_windows(image, points, window_size)

    windows = patches.astype(np.float64)

    try:
        feats = cnn.evalRGB(windows)
    except:
        print("ERROR windows.shape", windows.shape)
        raise

    feats = feats.reshape((windows.shape[0], feature_dim))
    X.extend(list(feats))


print("Clustering with KMeans: len(X)", len(X))

clusters = KMeans(n_clusters=n_clusters, verbose=False)
clusters.fit(np.array(X, dtype=np.float32))

pickle.dump(clusters, open(cluster_file, "wb"))

print("done")

100%|██████████| 15537/15537 [37:54<00:00,  6.83it/s]


Clustering with KMeans: len(X) 319862
done


### Generate sequence dataset with random walks

In [3]:
def get_grid_locations(grid_shape, stride, offsets, mask=None):
    
    if mask is not None:
        object_grid_locations = set()

        for y in range(grid_shape[0]):
            for x in range(grid_shape[1]):
                p = (offsets[0] + y * stride + 0.5 * stride, offsets[1] + x * stride + 0.5 * stride)
                w = extract_window(mask, p, stride)

                if np.sum(w) >= stride * stride * 0.3:
                    object_grid_locations.add((y, x))
        
        return object_grid_locations
    
    else:
        return [(y,x) for y in range(grid_shape[0]) for x in range(grid_shape[1])]
    
    
def generate_image_cluster_grid(image_file, image_scale, clusters, feature_extractor):
    # print("generate_image_sequences", image_file)

    pil_image = Image.open(image_file).convert('RGB')
    pil_image = pil_image.resize((int(round(pil_image.size[0] * image_scale)), int(round(pil_image.size[1] * image_scale))))
    image = np.array(pil_image)

    if image.shape[0] < window_size * 2 or image.shape[1] < window_size * 2:
        print("image too small, image_file")
        return None
            
    margin = window_size-stride
    grid_shape = (math.floor((image.shape[0] - margin) / stride), math.floor((image.shape[1] - margin) / stride))
    offsets = (round((image.shape[0] - grid_shape[0] * stride)/2), round((image.shape[1] - grid_shape[1] * stride)/2))

    grid_locations_set = get_grid_locations(grid_shape, stride, offsets)
    grid_locations_list = list(grid_locations_set)
    
    points = [(y*stride + stride/2 + offsets[0], x*stride + stride/2 + offsets[1]) for (y,x) in grid_locations_list]
        
    patches = extract_windows(image, points, window_size)
    windows = patches.astype(np.float64)

    # print(windows.shape)
    
    feats = cnn.evalRGB(windows)
    feats = feats.reshape((windows.shape[0], feature_dim))

    grid_cluster_ids = clusters.predict(feats)
        
    cluster_grid = np.full(grid_shape, -1, dtype=int)
    
    for i in range(len(grid_locations_list)):
        cluster_grid[grid_locations_list[i]] = grid_cluster_ids[i]
        
    return cluster_grid


def generate_image_sequences(image_cluster_grid, seq_count=walks_per_image):

    cluster_seqs = []
    
    grid_locations_set = set([(y,x) for y in range(image_cluster_grid.shape[0]) for x in range(image_cluster_grid.shape[1])])
    
    for i in range(seq_count):
        cluster_seq = []

        pos = None
        
        for t in range(walk_length):
            pos = next_pos(grid_locations_set, image_cluster_grid.shape, pos)
            cluster_seq.append(image_cluster_grid[pos])
            
        cluster_seqs.append([word_format_string.format(w) for w in cluster_seq])
        
    return cluster_seqs


def mask_locations(mask, stride, grid_shape, offsets):
    
    object_grid_locations = set()

    for y in range(grid_shape[0]):
        for x in range(grid_shape[1]):
            p = (offsets[0] + y * stride + 0.5 * stride, offsets[1] + x * stride + 0.5 * stride)
            w = extract_window(mask, p, stride)

            # print(np.sum(w))
            if np.sum(w) >= stride * stride * 0.3:
                object_grid_locations.add((y, x))

    return object_grid_locations

     
def generate_masked_image_sequences(image_file, mask_file, clusters, feature_extractor, seq_count=walks_per_image):

    pil_image = Image.open(image_file).convert('RGB')
    pil_image = pil_image.resize((int(round(pil_image.size[0] * image_scale)), int(round(pil_image.size[1] * image_scale))))
    image = np.array(pil_image)

    if image.shape[0] < window_size * 2 or image.shape[1] < window_size * 2:
        print("image too small, image_file")
        return None
            
    pil_mask = Image.open(mask_file).convert('1')
    pil_mask = pil_mask.resize((int(round(pil_mask.size[0] * image_scale)), int(round(pil_mask.size[1] * image_scale))))
    mask = np.array(pil_mask)
        
    margin = window_size-stride
    grid_shape = (math.floor((image.shape[0] - margin) / stride), math.floor((image.shape[1] - margin) / stride))
    offsets = (round((image.shape[0] - grid_shape[0] * stride)/2), round((image.shape[1] - grid_shape[1] * stride)/2))

    grid_locations_set = mask_locations(mask, stride, grid_shape, offsets)
    grid_locations_list = list(grid_locations_set)
    
    points = [(y*stride + stride/2 + offsets[0], x*stride + stride/2 + offsets[1]) for (y,x) in grid_locations_list]
        
    patches = extract_windows(image, points, window_size)
    windows = patches.astype(np.float64)

    feats = cnn.evalRGB(windows)
    feats = feats.reshape((windows.shape[0], feature_dim))

    grid_cluster_ids = clusters.predict(feats)
    grid_location_to_cluster_id = dict([(grid_locations_list[i], grid_cluster_ids[i]) for i in range(len(grid_locations_list))])
          
    cluster_seqs = []
    for i in range(seq_count):
        cluster_seq = []
        
        pos = None
        
        for t in range(walk_length):
            pos = next_pos(grid_locations_set, grid_shape, pos)
            cluster_seq.append(grid_location_to_cluster_id[pos])
            
        cluster_seqs.append([word_format_string.format(w) for w in cluster_seq])
  
    return cluster_seqs

In [13]:
clusters = pickle.load(open(cluster_file, "rb"))

image_cluster_grids = {}

for idx, image_file in tqdm(enumerate(image_files), total=len(image_files)):

    image_cluster_grid = generate_image_cluster_grid(image_file, image_scale, clusters, cnn)
    if image_cluster_grid is None:
        continue
    image_id = image_file.split('/')[-1].split('.')[0]
    image_cluster_grids[image_id] = image_cluster_grid    
    
np.save(image_cluster_grid_file, image_cluster_grids)   

print("done")
# print(np.load(image_cluster_grid_file, allow_pickle=True))

  0%|          | 65/15537 [00:13<51:33,  5.00it/s]  

image too small, image_file


  1%|          | 173/15537 [00:34<50:49,  5.04it/s]  

image too small, image_file


  1%|▏         | 213/15537 [00:42<40:26,  6.32it/s]

...

image too small, image_file


100%|█████████▉| 15527/15537 [54:32<00:01,  6.75it/s]

image too small, image_file


100%|██████████| 15537/15537 [54:33<00:00,  4.75it/s]

done


In [4]:
clusters = pickle.load(open(cluster_file, "rb"))
image_cluster_grids = np.load(image_cluster_grid_file, allow_pickle=True).item()

cluster_seqs = []
image_file_colummn = []

image_files = list(image_cluster_grids.keys())

for idx, image_file in tqdm(enumerate(image_files), total=len(image_files)):
    c_seqs = generate_image_sequences(image_cluster_grids[image_file])
    if c_seqs is None:
        continue
    cluster_seqs.extend(c_seqs)
    image_file_colummn.extend([image_file] * walks_per_image)

data_frame = pd.DataFrame({'words':cluster_seqs, 'file':image_file_colummn})

data_frame.to_csv(sequences_file)

print("done")

FileNotFoundError: [Errno 2] No such file or directory: 'image_cluster_grids_288_144_100.npy'

In [35]:
class callback(gensim.models.callbacks.CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print('epoch {}'.format(self.epoch))
        self.epoch += 1
              
def read_corpus(fname, tokens_only=False):
    data_frame = pd.read_csv(sequences_file,converters={"words": literal_eval})
    
    for index, row in data_frame.iterrows():
        if tokens_only:
            yield row['words']
        else:
            yield gensim.models.doc2vec.TaggedDocument(row['words'], [index])

train_corpus = list(read_corpus(sequences_file))
print(train_corpus[:2])

model = gensim.models.doc2vec.Doc2Vec(vector_size=256, epochs=30)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs, callbacks=[callback()])

model.save(doc2vec_file)

print("done")

[TaggedDocument(words=['687', '202', '286', '091', '286', '091', '091', '091', '837', '837'], tags=[0]), TaggedDocument(words=['091', '806', '806', '806', '286', '286', '286', '806', '266', '687'], tags=[1])]
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
done


In [36]:
clusters = pickle.load(open(cluster_file, "rb"))
model = gensim.models.doc2vec.Doc2Vec.load(doc2vec_file)

data_frame = pd.read_csv(sequences_file, converters={"words": literal_eval})

test_image_files = glob("dataset_100/test/*/*.jpg")
test_mask_files = glob("dataset_100/test/*/*.mask.png")

test_image_files.sort()
test_mask_files.sort()


correct = 0
total = 0

for i in range(len(test_image_files)):
    
    image_correct = 0
    image_total = 0
    
    image_file = test_image_files[i]
    mask_file = test_mask_files[i]

    print("test", image_file)

    c_seqs = generate_masked_image_sequences(image_file, mask_file, clusters, cnn, seq_count=100)

    if c_seqs is None:
        continue
        
    vectors = [[model.infer_vector(s)] for s in c_seqs]

    for v in vectors:
        similar = model.docvecs.most_similar(v, topn=10)
        #print('similar', similar)

        for s in similar:
            f = data_frame.loc[s[0],'file']
            
            a = image_file.split('/')[-2]
            b = image_id_to_class[f]
           
            if a == b:
                image_correct += 1
                correct += 1
           
            image_total += 1
            total += 1
            
    print("score", image_correct/image_total)
    
print("final score", correct/total)

test dataset_100/test/airplane/35b11a04c24db20c.jpg
score 0.73
test dataset_100/test/airplane/4ad0f079b979be5d.jpg
score 0.291
test dataset_100/test/airplane/839ce813ca97084c.jpg
score 0.386
test dataset_100/test/airplane/93b5bf58149adefd.jpg
score 0.909
test dataset_100/test/airplane/9dc879c35a26d2d3.jpg
score 0.721
test dataset_100/test/airplane/a48f1d15812036fa.jpg
score 0.297
test dataset_100/test/airplane/b6ac22d7db1769ee.jpg
score 0.355
test dataset_100/test/airplane/d5422871fd63b8b8.jpg
score 0.837
test dataset_100/test/airplane/e95bc413d4b748ba.jpg
score 0.323
test dataset_100/test/airplane/fbe835c5944f93e5.jpg
score 0.293
test dataset_100/test/car/455c29cd8db5b225.jpg
score 0.24
test dataset_100/test/car/56d1d8aca15ae219.jpg
score 0.413
test dataset_100/test/car/89297009b1d18663.jpg
score 0.364
test dataset_100/test/car/9bac3f90244fef7e.jpg
score 0.271
test dataset_100/test/car/a051a80f600fd919.jpg
score 0.373
test dataset_100/test/car/b30bc1fe86057942.jpg
score 0.33
test data